# 정리노트 - 데이터 마이닝

### 트레이닝, 테스트 데이터 셋 나누기

In [ ]:
#트레이닝, 테스트 데이터 셋 나누기
from sklearn.model_selection import train_test_split

train_test_split()

### 의사결정나무 (모델평가 포함)

In [ ]:
#의사결정나무
#참고 : https://datascienceschool.net/03%20machine%20learning/12.01%20%EC%9D%98%EC%82%AC%EA%B2%B0%EC%A0%95%EB%82%98%EB%AC%B4.html
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

train_accuracy = []
predict_accuracy = []
depth_range = range(3, 10)
for max_depth in depth_range:
    model = DecisionTreeClassifier(criterion="entropy", 
                                   max_depth=max_depth).fit(X_train, y_train)
    this_train_acc = accuracy_score(y_train, model.predict(X_train))
    this_test_acc = accuracy_score(y_test, model.predict(X_test))
    train_accuracy.append(this_train_acc)
    predict_accuracy.append(this_test_acc)
    auc_score = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
    print("max_depth : {}".format(max_depth))
    print("auc_score : {}".format(roc_score))
    print("train_acc - test_acc : {}".format(this_train_acc - this_test_acc))
    print(classification_report(y_test, model.predict(X_test))+"\n\n")

plt.xlabel("max_depth")
plt.ylabel("accuracy")
plt.plot(depth_range, train_accuracy, label="train_accuracy")
plt.plot(depth_range, predict_accuracy, label="predict_accuracy")
plt.legend()
plt.show()

maxima_idx = np.argmax(predict_accuracy)
print("가장 높은 predict_accuracy를 보인 depth : {}".format(depth_range[maxima_idx]))

In [ ]:
#4개 depth를 가진 모델을 가지고 ROC커브로 최적의 threshold를 찾아 정확도를 높여보자.
from sklearn.metrics import roc_curve
model = DecisionTreeClassifier(criterion="entropy", 
                               max_depth=4).fit(X_train, y_train)

fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])
roc = pd.DataFrame({
    "FPR": fpr, 
    "TPR": tpr, 
    "Threshold": thresholds,
    "TPR-FPR" : tpr-fpr
})
display(roc)
plt.plot(roc["FPR"], roc["TPR"])
maxima = np.argmax(roc["TPR-FPR"])
print("TPR - FPR 최적 threshold값 : {}".format(roc.iloc[maxima, 2]))

In [ ]:
from sklearn.preprocessing import Binarizer

custom_threshold = roc.iloc[maxima, 2]
pred_proba_flatten = model.predict_proba(X_test)[:,1].reshape(-1,1)
binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_flatten) 
custom_predict = binarizer.transform(pred_proba_flatten)
print("오리지날")
print(classification_report(y_test, model.predict(X_test))+"\n\n")

print("threshold : {}".format(custom_threshold))
print(classification_report(y_test, custom_predict)+"\n\n")

#더 낮아지네???

### 랜덤 포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# n_estimators : 모델에서 사용할 트리 갯수(학습시 생성할 트리 갯수)
# criterion : 분할 품질을 측정하는 기능 (default : gini)
# max_depth : 트리의 최대 깊이
# min_samples_split : 내부 노드를 분할하는데 필요한 최소 샘플 수 (default : 2)
# min_samples_leaf : 리프 노드에 있어야 할 최소 샘플 수 (default : 1)
# min_weight_fraction_leaf : 
#               min_sample_leaf와 같지만 가중치가 부여된 샘플 수에서의 비율
# max_features : 각 노드에서 분할에 사용할 특징의 최대 수
# max_leaf_nodes : 리프 노드의 최대수
# min_impurity_decrease : 최소 불순도
# min_impurity_split : 나무 성장을 멈추기 위한 임계치
# bootstrap : 부트스트랩(중복허용 샘플링) 사용 여부
# oob_score : 일반화 정확도를 줄이기 위해 밖의 샘플 사용 여부
# n_jobs :적합성과 예측성을 위해 병렬로 실행할 작업 수
# random_state : 난수 seed 설정
# verbose : 실행 과정 출력 여부
# warm_start : 이전 호출의 솔루션을 재사용하여 합계에 더 많은 견적가를 추가
# class_weight : 클래스 가중치

In [ ]:
#랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix

model = RandomForestClassifier(n_estimators=500, max_depth=4, 
                               criterion="entropy", oob_score=True)
model.fit(X_train, y_train["survived"])
train_acc = accuracy_score(y_train, model.predict(X_train))
test_acc = accuracy_score(y_test, model.predict(X_test))
auc_score = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])

print("oob_score : {}".format(model.oob_score_))
print("train_acc : {}".format(train_acc))
print("test_acc : {}".format(test_acc))
print("auc_score : {}".format(roc_score))
print("train_acc - test_acc : {}".format(this_train_acc - this_test_acc))
print(confusion_matrix(y_test, model.predict(X_test)))
print(confusion_matrix(y_test, model.predict(X_test), normalize="true"))
print(classification_report(y_test, model.predict(X_test))+"\n\n")

fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])
roc = pd.DataFrame({
    "FPR": fpr, 
    "TPR": tpr, 
    "Threshold": thresholds,
    "TPR-FPR" : tpr-fpr
})
display(roc)
plt.plot(roc["FPR"], roc["TPR"])
maxima = np.argmax(roc["TPR-FPR"])
print("TPR - FPR 최적 threshold값 : {}".format(roc.iloc[maxima, 2]))

### 그라디언트 부스팅

![title](https://github.com/hrdkdh/adp-study/blob/main/images/ensenble.png?raw=true)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=500)
model.fit(X_train, y_train["survived"])

train_acc = accuracy_score(y_train, model.predict(X_train))
test_acc = accuracy_score(y_test, model.predict(X_test))
auc_score = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])

print("train_acc : {}".format(train_acc))
print("test_acc : {}".format(test_acc))
print("auc_score : {}".format(roc_score))
print("train_acc - test_acc : {}".format(this_train_acc - this_test_acc))
print(confusion_matrix(y_test, model.predict(X_test)))
print(confusion_matrix(y_test, model.predict(X_test), normalize="true"))
print(classification_report(y_test, model.predict(X_test))+"\n\n")

fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])
roc = pd.DataFrame({
    "FPR": fpr, 
    "TPR": tpr, 
    "Threshold": thresholds,
    "TPR-FPR" : tpr-fpr
})
display(roc)
plt.plot(roc["FPR"], roc["TPR"])
maxima = np.argmax(roc["TPR-FPR"])
print("TPR - FPR 최적 threshold값 : {}".format(roc.iloc[maxima, 2]))

### k-means

최적의 군집갯수를 찾는 방법은 엘보우 방법과 실루엣 방법이 있다.<br>
엘보우 방법은 군집의 갯수만큼 군집 중심과 군집에 속한 개체들 간의 거리, SSE를 계산해 특정 군집갯수에서 SSE가 확 줄어든다면 그 이전 갯수를 최적의 군집갯수로 정하는 것이다. (팔꿈치가 꺽이는 부분을 최적 갯수로 함(아래 참고)<br><br>
k-means 클러스터링은 클러스터내 오차제곱합(SSE)의 값이 최소가 되도록 클러스터의 중심을 결정해나가는 방법이라고 했습니다. 만약 클러스터의 개수를 1로 두고 계산한 SSE 값과, 클러스터의 개수를 2로 두고 계산한 SSE 값을 비교했을 때, 클러스터의 개수를 2로 두고 계산한 SSE 값이 더 작다면, 1개의 클러스터보다 2개의 클러스터가 더 적합하다는 것을 짐작할 수 있습니다.
이런 식으로 클러스터의 개수를 늘려나가면서 계산한 SSE를 그래프로 그려봅니다. SSE의 값이 점점 줄어들다가 어느 순간 줄어드는 비율이 급격하게 작아지는 부분이 생기는데, 그래프 모양을 보면 팔꿈치에 해당하는 바로 그 부분이 우리가 구하려는 최적의 클러스터 개수가 됩니다. elbow가 우리말로 팔꿈치라는거 다 아실거고요<br><br>
![title](https://raw.githubusercontent.com/hrdkdh/adp-study/main/images/elbow.png)
<br><br>
실루엣 방법은 한 클러스터 안에 있는 데이터들이 다른 클러스터와 비교해서 얼마나 비슷한가를 나타내는 실루엣 계수를 이용한다.<br>
클러스터 안의 거리가 짧을수록 좋고(cohesion, 응집도), 다른 클러스터와의 거리가 멀수록 좋다.(separation, 분리도)<br>
아래 그림에서 a는 응집도, b는 분리도이다.<br><br>
![title](https://raw.githubusercontent.com/hrdkdh/adp-study/main/images/silhouette.png)<br>
실루엣 계수는 -1~1사이의 값을 가지며 1에 가까울수록 잘 부합하는 데이터이다. 실루엣 계수가 0이면 지금클러스터나 다른 클러스터 어디에 있든 상관없음을 의미한다.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

#엘보우 기법을 사용해 최적 갯수를 찾는다.
sse = []
cluster_range = range(2, 16)
for i in cluster_range:
    print("{}개로 클러스터링 중...".format(i), end="\r")
    kmeans = KMeans(n_clusters = i)
    kmeans.fit(df3)
    sse.append(kmeans.inertia_)
print("\n클러스터링 완료")

plt.figure(dpi=200)
plt.plot(cluster_range, sse, marker="o")
plt.xlabel("Cluster n")
plt.xlabel("SSE")
plt.show()
#대략 6 즈음에서 완만해 지는 듯 하다. 최적 갯수는 6으로 결정!

#6개로 다시 클러스터링해 본 다음 중심점을 본다.
kmeans = KMeans(n_clusters = 6)
kmeans.fit(df3)
kmeans.cluster_centers_

#라벨링을 한다
labels = kmeans.predict(df3)
df4 = df3.copy()
df4["clust"] = labels
df4

#각 군집별 레코드의 갯수는?
for i in range(0, 6):
    print("{}번 군집에 속한 레코드의 갯수 : {}".format(i, 
                                           len(df4.loc[(df4.clust == i)])))